In [1]:
# Colabis kasutamiseks
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/thesis/')

!pip install estnltk==1.7.4
!pip install evaluate
!pip install seqeval
!pip install nervaluate

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-n

In [5]:
from modules.data_processing import DatasetProcessor
from modules.bert_data_processing import BERTDataProcessor
from modules.bert_evaluator import BERTEvaluator
ALL_TAGS = DatasetProcessor.ALL_TAGS
TAG2IDX = DatasetProcessor.TAG2IDX
IDX2TAG = DatasetProcessor.IDX2TAG
from transformers import BertForTokenClassification, Trainer, TrainingArguments, AutoModelForTokenClassification
import json
from datasets import Dataset

In [6]:
def load_test_data(path):
  with open(path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    ids, tokens, tags = [], [], []
    id = 0
    for document in data['documents']:
      for sentence in document['sentences']:
        sentence_tokens = []
        sentence_tags = []
        for word in sentence['words']:
          token = word['word']
          tag = word['ner_1']

          match tag:
            case 'B-GPE':
              tag = 'B-GEP'
            case 'I-GPE':
              tag = 'I-GEP'
            case 'B-EVENT':
              tag = 'B-EVE'
            case 'I-EVENT':
              tag = 'I-EVE'

          if tag not in ALL_TAGS:
            tag = 'O'

          sentence_tokens.append(token)
          sentence_tags.append(TAG2IDX[tag])


        ids.append(id)
        tokens.append(sentence_tokens)
        tags.append(sentence_tags)
        id += 1
    return Dataset.from_dict({'id': ids, 'tokens': tokens, 'tags': tags})

In [7]:
estner_new = load_test_data('data/estner/EstNER_new_test.json')

In [8]:
estner_v1 = load_test_data('data/estner/EstNER_v1_test.json')

In [9]:
model = AutoModelForTokenClassification.from_pretrained('results/EstBERT/edt/checkpoint-4614')
bert_processor = BERTDataProcessor('results/EstBERT/edt/checkpoint-4614')

In [10]:
label2id = model.config.label2id
def convert_to_model_id(example):
  example['tags'] = [IDX2TAG[tag] for tag in example['tags']]
  example['tags'] = [label2id.get(tag) for tag in example['tags']]
  return example

estner_new = estner_new.map(convert_to_model_id)
estner_v1 = estner_v1.map(convert_to_model_id)

estner_new = bert_processor.tokenize_dataset(estner_new)
estner_v1 = bert_processor.tokenize_dataset(estner_v1)

Map:   0%|          | 0/890 [00:00<?, ? examples/s]

Map:   0%|          | 0/1907 [00:00<?, ? examples/s]

Map:   0%|          | 0/890 [00:00<?, ? examples/s]

Map:   0%|          | 0/1907 [00:00<?, ? examples/s]

In [37]:
tags = [i for i in range(len(model.config.id2label))]

for a in model.config.id2label:
  tags[a] = model.config.id2label[a]
evaluator = BERTEvaluator(all_tags=tags)

training_args = TrainingArguments(
            report_to='none',
            output_dir='./results',
            #per_device_eval_batch_size=16
        )
trainer = Trainer(
            model=model,
            processing_class=bert_processor.tokenizer,
            data_collator=bert_processor.data_collator,
            compute_metrics=evaluator.compute_metrics,
            args=training_args
        )

In [38]:
estner_new_results = evaluator.evaluate_and_print(estner_new, trainer)

Hindan testandmestikul..


Seqeval tulemused
EVE {'precision': 0.3472222222222222, 'recall': 0.5434782608695652, 'f1': 0.423728813559322, 'number': 46}
GEP {'precision': 0.6036363636363636, 'recall': 0.4783861671469741, 'f1': 0.5337620578778135, 'number': 347}
LOC {'precision': 0.1382636655948553, 'recall': 0.6142857142857143, 'f1': 0.2257217847769029, 'number': 70}
ORG {'precision': 0.5826558265582655, 'recall': 0.5955678670360111, 'f1': 0.5890410958904109, 'number': 361}
PER {'precision': 0.5448460508701473, 'recall': 0.8409090909090909, 'f1': 0.6612510154346061, 'number': 484}
PROD {'precision': 0.25263157894736843, 'recall': 0.2302158273381295, 'f1': 0.24090338770388958, 'number': 417}
overall_precision 0.44196843082636955
overall_recall 0.5518840579710145
overall_f1 0.4908481567414282
overall_accuracy 0.9066625412541254

Nervaluate tulemused
Strict {'correct': 952, 'incorrect': 440, 'partial': 0, 'missed': 333, 'spurious': 819, 'possible': 1725, 'actual': 2211, 'precision': 0.43057440072365444, 'recall': 0.

In [39]:
estner_v1_results = evaluator.evaluate_and_print(estner_v1, trainer)


Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.11678832116788321, 'recall': 0.37209302325581395, 'f1': 0.17777777777777778, 'number': 43}
GEP {'precision': 0.5574229691876751, 'recall': 0.615146831530139, 'f1': 0.5848640705363704, 'number': 647}
LOC {'precision': 0.12372881355932204, 'recall': 0.73, 'f1': 0.21159420289855074, 'number': 100}
ORG {'precision': 0.3748344370860927, 'recall': 0.5705645161290323, 'f1': 0.45243804956035166, 'number': 992}
PER {'precision': 0.6051428571428571, 'recall': 0.7258396161754627, 'f1': 0.6600186974135245, 'number': 1459}
PROD {'precision': 0.05056179775280899, 'recall': 0.2, 'f1': 0.08071748878923767, 'number': 135}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
overall_precision 0.4085179526355997
overall_recall 0.6335900473933649
overall_f1 0.4967487227124942
overall_accuracy 0.8892503835370047

Nervaluate tulemused
Strict {'correct': 2139, 'incorrect': 1007, 'partial': 0, 'missed': 230, 'spurious': 2188, 'possible': 3376, 'actual': 5334, 'pr